PyFluent Simulation Hackathon
---
The purpose of the following notebook is to provide a high level introduction to the workings of a fluid dynamics simulation. It will guide you through a step-by-step process of the preparation, setup, running and post-processing of a simulation. Primarily this tutorial is deisgned to promote experimentation and iteration. 

In this case we will be simulating an internal flow through different venturis.

**Note:** Be sure to restart the kernel between every time you run the script to avoid bugs.

In [ ]:
import os , zipfile

home_directory = os.path.expanduser("~")
zip_file_path = f'{home_directory}/hackathon.zip'
venturi_dir = f'{home_directory}/venturis/'

os.makedirs(venturi_dir, exist_ok=True)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.startswith('hackathon/venturi/') and not file.endswith('/'):
            with zip_ref.open(file) as source, open(os.path.join(venturi_dir, os.path.basename(file)), 'wb') as target:
                    target.write(source.read())
        elif file.startswith('hackathon/') and file.endswith('.py'):
            with zip_ref.open(file) as source, open(os.path.join(home_directory+'/', os.path.basename(file)), 'wb') as target:
                    target.write(source.read())
               
from SimpleSim import SimpleSim
import FileManager as fm

try:
    fm.delete_file('hackathon.zip')
except:
    pass

The follow block of code defines key variables which shape the nature of the simulation: 
    
- **geometry_file:** Select the name of geometry file that will be used in the simulation, in this case you can choose between different venturis by replacing the number between *venturi* and *.pmdb* with *0, 2, 4, 6, 8* or *10*.

- **simulation_model:** Select the model type. Choose between *laminar, k-epsilon, k-omega, inviscid*

- **inlet_velocity:** Velocity in m/s. Choose any number.

- **fluid_material:** Material of fluid being simulated. Try *air, water-liquid, mercury, hydrogen* or *engine-oil*
- **contour_field:** A contour plot is essentially like a heat/color map, the contour field is the value that is being plotted. The options are *velocity-magnitude, pressure* and *density*.

- **xyplot_field:** Y-axis parameter, same options as for contour_field. 

In [ ]:
geometry_file = 'venturi6.pmdb' 
simulation_model = 'inviscid' 
inlet_velocity = 5
fluid_material = 'meury'
contour_field = 'velocity-magnitude' 
xyplot_field = 'velocity-magnitude' 

#create a PyFluent "instance" -- equivalent of opening a new window of an app 
sim = SimpleSim(geometry_file , simulation_model , inlet_velocity , fluid_material)

In [ ]:
#copy the geometry file into the PyFluent instance so that it can be accessed 
file_path = fm.get_file_path(geometry_file)
fm.copy_file(file_path , sim.instance)

Meshing
--

In [ ]:
#import the geometry into the Fluent meshing app and generate a standard mesh
sim.import_geometry()
sim.generate_mesh()
sim.show_mesh()

In [ ]:
#now that the mesh has been generated, switch from the, meshing app to the solving app
sim.solver = sim.meshing.switch_to_solver()

Solver
--

In [ ]:
#initialize the simulation by defining the physics parameters -- including the model and the inlet velocity which were defined at the beginning of the notebook. Then begin the calculation. 
sim.setup_sim()
sim.simulate()

Post-processing
--

In [ ]:
#In this block we setup and plot our contour plot. 
plane_surface_name = 'symmetry-plane'
#check if surface already exists to prevent error due to overwriting 
try: 
    sim.solver.tui.surface.delete_surface(plane_surface_name)
except:
    pass
#create a plane surface, parallel to the xy-plane, with a z value of 125mm (middle of geometry) 
sim.solver.tui.surface.plane_surface(plane_surface_name,"xy-plane","125")

#combine the parameters and data to plot 
sim.contours(contour_field , plane_surface_name)

In [ ]:
#In this block we setup and plot our contour plot.
line_surface_name = 'centerline'
#check if surface already exists to prevent error due to overwriting
try: 
    sim.solver.tui.surface.delete_surface(line_surface_name)
except:
    pass
#create a line surface, which conects the point [-75,0,125] and [75,0,125] (also center of geometry). This essentially serves as the x-axis to our plot 
sim.solver.tui.surface.line_surface(line_surface_name,"-75","0","125","75","0","125")

#combine the parameters and data to plot 
sim.xyplot(xyplot_field , line_surface_name)


In [ ]:
#from ansys.fluent.visualization.pyvista import  Graphics
#graphics_session = Graphics(sim.solver)
#pathlines1 = graphics_session.Pathlines["pathlines-1"]
#pathlines1.field = "velocity-magnitude"
#pathlines1.surfaces_list = ['inlet']
#pathlines1.display()

In [ ]:
#This line is put here for convenience, it deletes the instance and all of its data so the Workspace is not cluttered. Will not work if the kernal was restarted. 

fm.delete_dir(sim.instance)